# 线性回归的简洁实现 

---

## 导包，生成数据

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## 调用框架中现有的API来读取数据

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays) # * 是元组拆包符
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.3271,  0.4994],
         [-1.3543,  1.1083],
         [-0.2150, -0.9627],
         [-1.0403, -0.4282],
         [ 0.8348,  0.8511],
         [ 0.7433,  1.0709],
         [ 1.0247, -0.9685],
         [-0.2875, -0.0479],
         [-0.2987, -0.7989],
         [ 0.8916, -0.1745]]),
 tensor([[ 3.1539],
         [-2.2723],
         [ 7.0440],
         [ 3.5824],
         [ 2.9926],
         [ 2.0377],
         [ 9.5504],
         [ 3.8003],
         [ 6.2963],
         [ 6.5932]])]

## 使用框架的预定义好的层

In [3]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1)) #指定输入输出

## 初始化模型参数

In [9]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)


tensor([0.])

## 计算均方误差使用的是MSELoss类，也叫作平方L2范数

In [5]:
loss = nn.MSELoss()

## 实例化SGD实例

In [6]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 训练

In [7]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000286
epoch 2, loss 0.000101
epoch 3, loss 0.000102


In [11]:
print(f'w的估计误差：{true_w - net[0].weight.reshape(true_w.shape)}')
print(f'b的估计误差：{true_b - net[0].bias}')

w的估计误差：tensor([ 1.9890, -3.3938], grad_fn=<SubBackward0>)
b的估计误差：tensor([4.2000], grad_fn=<RsubBackward1>)
